In [1]:
import pandas

In [2]:
import os
import pandas as pd

# Get the parent directory of the current directory
parent_directory = os.path.dirname(os.getcwd())

# Set the data_directory to the 'kaggle' folder in the parent directory
data_directory = os.path.join(parent_directory, "data", "kaggle","raw")

# Create an empty dictionary to store DataFrames
dataframes = {}

# Iterate through all files in the data_directory
for filename in os.listdir(data_directory):
    # Check if the file is a CSV file
    if filename.endswith(".csv"):
        file_path = os.path.join(data_directory, filename)

        # Read the first 20 lines of the CSV file as a DataFrame
        df = pd.read_csv(file_path)

        # Add the DataFrame to the dictionary with the filename as the key
        dataframes[filename] = df

# Print the DataFrames
for filename, df in dataframes.items():
    print(f"Data for {filename}:\n")



/home/dimi/miniconda3/envs/imdb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Data for links.csv:

Data for movies_metadata.csv:

Data for ratings_small.csv:

Data for ratings.csv:

Data for keywords.csv:

Data for credits.csv:

Data for links_small.csv:



In [3]:
dataframes.keys()

dict_keys(['links.csv', 'movies_metadata.csv', 'ratings_small.csv', 'ratings.csv', 'keywords.csv', 'credits.csv', 'links_small.csv'])

In [4]:
dataframes['keywords.csv'].tail(4)

,id,keywords
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]
46418,461257,[]


In [5]:
dataframes['movies_metadata.csv']['genres'].iloc[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [6]:
dataframes['movies_metadata.csv'].columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [7]:
df0=dataframes['movies_metadata.csv'][['original_title','genres','overview']]

In [8]:
df0

,original_title,genres,overview
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",Just when George Banks has recovered from his ...
...,...,...,...
45461,رگ خواب,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",Rising and falling between a man and woman.
45462,Siglo ng Pagluluwal,"[{'id': 18, 'name': 'Drama'}]",An artist struggles to finish his work while a...
45463,Betrayal,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...","When one of her hits goes wrong, a professiona..."
45464,Satana likuyushchiy,[],"In a small town live two brothers, one a minis..."


In [9]:
import ast

# Create a function to extract the genre names from a list of genres
def extract_genres(genre_list):
    genres = []
    for genre in genre_list:
        genres.append(genre['name'])
    return genres

# Convert the 'genres' string to a list of dictionaries
df0['genres'] = df0['genres'].apply(lambda x: ast.literal_eval(x))

# Extract the genre names from the list of dictionaries
df0['genre_list'] = df0['genres'].apply(lambda x: extract_genres(x))


/tmp/ipykernel_5243/1439879981.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df0['genres'] = df0['genres'].apply(lambda x: ast.literal_eval(x))
/tmp/ipykernel_5243/1439879981.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df0['genre_list'] = df0['genres'].apply(lambda x: extract_genres(x))


In [13]:
processed_df = df0[['original_title','genre_list', 'overview']]

In [17]:
#remove rows with null values
processed_df = processed_df.dropna()
#remove rows with empty strings
processed_df = processed_df[processed_df['overview'] != '']
#remove rows with no data
processed_df = processed_df[processed_df['overview'] != 'No overview found.']
#remove rows with no data
processed_df = processed_df[processed_df['overview'] != 'No Overview']
#remove rows with no genre  
processed_df = processed_df[processed_df['genre_list'] != '[]']

In [20]:
processed_df['genre_list'].iloc[-1]

[]

In [31]:
import os

# Create the 'processed' directory if it doesn't already exist
if not os.path.exists('../data/processed'):
    os.makedirs('../data/processed')

# Save the DataFrame to a CSV file in the 'processed' directory
processed_df.to_csv('../data/processed/processed_data.csv')


processed dataframe is no in data/processed

In [32]:
df=pd.read_csv('../data/processed/processed_data.csv',index_col=0)

In [40]:
duplicates = df[df.duplicated(subset='overview', keep=False)].sort_values(by='overview')

In [41]:
duplicates

,genre_list,overview
original_title,,
Slaves of New York,['Drama'],
Sur,[],
Snow days,"['Comedy', 'Romance']",
Αυστηρώς Κατάλληλο,['Comedy'],
Iceland,[],
...,...,...
Charlotte's Web,"['Comedy', 'Family', 'Fantasy']",Wilbur the pig is scared of the end of the sea...
Camille Claudel 1915,['Drama'],"Winter, 1915. Confined by her family to an asy..."
Camille Claudel 1915,['Drama'],"Winter, 1915. Confined by her family to an asy..."


In [19]:
# transform 'genre_liste' in binary encoded columns
df = df.join(df['genre_list'].str.join('|').str.get_dummies())



In [20]:
df

,genre_list,overview,,',",",A,C,D,F,H,...,l,m,n,o,r,s,t,u,v,y
original_title,,,,,,,,,,,,,,,,,,,,,
Toy Story,"['Animation', 'Comedy', 'Family']","Led by Woody, Andy's toys live happily in his ...",1,1,1,1,1,0,1,0,...,1,1,1,1,0,0,1,0,0,1
Jumanji,"['Adventure', 'Fantasy', 'Family']",When siblings Judy and Peter discover an encha...,1,1,1,1,0,0,1,0,...,1,1,1,0,1,1,1,1,1,1
Grumpier Old Men,"['Romance', 'Comedy']",A family wedding reignites the ancient feud be...,1,1,1,0,1,0,0,0,...,0,1,1,1,0,0,0,0,0,1
Waiting to Exhale,"['Comedy', 'Drama', 'Romance']","Cheated on, mistreated and stepped on, the wom...",1,1,1,0,1,1,0,0,...,0,1,1,1,1,0,0,0,0,1
Father of the Bride Part II,['Comedy'],Just when George Banks has recovered from his ...,0,1,0,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Strange Days,"['Crime', 'Drama', 'Science Fiction', 'Thriller']",Set in the year 1999 during the last days of t...,1,1,1,0,1,1,1,0,...,1,1,1,1,1,0,1,0,0,0
Les parapluies de Cherbourg,"['Drama', 'Romance']",This simple romantic tragedy begins in 1957. G...,1,1,1,0,0,1,0,0,...,0,1,1,1,1,0,0,0,0,0
The Tie That Binds,['Thriller'],John Netherwood and his wife Leann are fugitiv...,0,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
